# Inventário

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [1]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz

In [2]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [3]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
from turfpy.misc import nearest_point_on_line
from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [5]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [6]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [7]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [8]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [9]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [10]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [11]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [13]:
!cat ../.env

STAGE=PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [74]:
filename='Cópia de Drenagem ERS-122 Ok_2'
road_name='122'

Escolhendo a aba da planilha do Excel

In [75]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

Testando planilha carregada do Excel

In [76]:
print(ws['A1'].value)

km


Carregando os valores no script

In [77]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    if obj['Rodovia'] is not None:           
   
        obj['img'] = [a for a in ws._images if a.anchor._from.row == index]

        try:
            for a in obj['img']:
                a.name = 'CSG'
#             obj['img'][0].name='Verso'
        except:
            pass

        
    values.append(obj)

Teste de valores importados da planilha

In [78]:
len(values),values[2]

(190,
 {'km': None,
  'km final': None,
  'km de Projeto': None,
  'km final de Projeto': None,
  'Latitude_Inicial': -29.452055555555557,
  'Longitude_Inicial': -51.35268611111111,
  'Latitude_Final': -29.456802777777778,
  'Longitude_Final': -51.35292777777778,
  'Status': 'Identificado',
  'Equipe/Empreiteira': 'Cadastro incial',
  'Encontrado em': None,
  'Executado em': None,
  'Sentido': 'Central',
  'Classe': 'Drenagem',
  'Faixa': 'Não se aplica',
  'Rodovia': 'ERS-122',
  'Tipo': 'Meio-fio',
  'Extensão\n(m)': None,
  'Estado de conservação': 'Bom',
  'Observações': None,
  'Foto_1': None,
  'Data Foto_1': None,
  'Tipo Foto_1': None,
  'Descrição Foto_1': None,
  'Foto_2': None,
  'Data Foto_2': None,
  'Tipo Foto_2': None,
  'Descrição Foto_2': None,
  'Foto_3': None,
  'Data Foto_3': None,
  'Tipo Foto_3': None,
  'Descrição Foto_3': None,
  'Foto_4': None,
  'Data Foto_4': None,
  'Tipo Foto_4': None,
  'Descrição Foto_4': None,
  'Foto_5': None,
  'Data Foto_5': None,
  '

Definindo valores da importação

In [2]:
company = Company.objects.get(name='Caminhos da Serra Gaúcha')

#Classe Drenagem
occurrence_type=OccurrenceType.objects.get(uuid='0a7b03e7-38c0-4212-af50-c2020ff319b6')

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
# step=ApprovalStep.objects.get(approval_flow__company=company, name='Em Elaboração')
firm=Firm.objects.get(uuid='0edd2ecc-8d5e-4444-898b-0069a3e9c354')
company,occurrence_type,firm,road_name

NameError: name 'road_name' is not defined

Definir a Rodovia

In [80]:
# road=Road.objects.get(company=company,name__icontains="470")
# items=list(road.marks.items())
# first_km=items[0][1]['km']
# last_km=items[-1][1]['km']
# print(first_km,last_km)

In [81]:
print('Km inicial',' | ','km',' | ','km final')
for i,a in enumerate(tqdm(values)):
    road,_,point,km =point_to_km(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']),road_name)
    _,_,_,end_km =point_to_km(float(a['Longitude_Final']),float(a['Latitude_Final']),road_name)
    print(km,end_km)
# road,_,point,km =point_to_km(float(str(values[2]['Longitude_Inicial'])),float(str(values[2]['Latitude_Inicial'])),road)
road,point,km

Km inicial  |  km  |  km final


  0%|          | 0/190 [00:00<?, ?it/s]

29.965 29.798
29.776 29.261
29.242 28.716


KeyboardInterrupt: 

Criando itens de Inventário

In [82]:
objects=[]


for index, a in enumerate(tqdm(values)):

    road,_,point,km =point_to_km(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']),road_name)
    _,_,_,end_km =point_to_km(float(a['Longitude_Final']),float(a['Latitude_Final']),road_name)
    if road_name not in road.name:
        print(f"Rodovia {road} - Km:{km} - Linha: {index+2}")
    try:

        objects.append((Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    road_name=road.name,
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
#                     found_at=datetime.datetime('2023-05-08'),
                    km=float(km),
                    end_km_manually_specified= True,
                    end_km=float(end_km),
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        'localdrenagem': get_value('Tipo', occurrence_type, a['Tipo']) if a['Tipo'] is not None and a['Tipo'] != 'Valeta Proteção de Corte' else 'Valeta de Proteção de Corte',
                        'state': get_value('Estado de conservação', occurrence_type, a['Estado de conservação']) if a['Estado de conservação'] is not None else None,
                        'extension': a['Extensão\n(m)'] if a['Extensão\n(m)'] is not None else '',                                                   
                    }, 
                    ),a['img']))
    except Exception as e:

        print(f'Arquivo"{filename}" - Rodovia {road} - Km:{km} - Linha: {index+2} - {e}')
        print(objects.form_data)

  0%|          | 0/190 [00:00<?, ?it/s]

Salvando itens de Inventário no sistema e imprimindo os seriais

In [83]:
objects[0][0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f4f987def90>,
 'uuid': UUID('104a481c-8c5c-4381-a2ce-9ba1ed773a00'),
 'number': '',
 'company_id': UUID('4edb7778-e350-4e77-8e1e-de5f87b1da7f'),
 'road_name': 'ERS-122',
 'road_id': 2059,
 'km': 29.965,
 'end_km': 29.798,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f4f8d91ff40>,
 'direction': '4',
 'lane': '99',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': UUID('0edd2ecc-8d5e-4444-898b-0069a3e9c354'),
 'occurrence_type_id': UUID('0a7b03e7-38c0-4212-af50-c2020ff319b6'),
 'form_data': {'localdrenagem': '9', 'state': '1', 'extension': ''},
 'form_metadata': {},
 'executed_at': None,
 'created_at': None,
 'updated_at': None,
 'found_at': datetime.datetime(2023, 8, 11, 13, 0, 44, 220179, tzinfo=<UTC>),
 'due_at': None,
 'status_id': UUID('c4b0df55-dc0e-47ba-83e7-86080047a8d5'),
 'job_id': None,
 'parent_id': No

In [84]:
for reporting, images in tqdm(objects):
    reporting.save()
    print(reporting.number)

  0%|          | 0/190 [00:00<?, ?it/s]

Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27684
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27685
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27686
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27687
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27688
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27689
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27690
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27691
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27692
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27693
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27694
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27695
Searching KM on direction...
Found KM on Road 2059: 

CSG-Inv-2023.27765
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27766
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27767
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27768
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27769
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27770
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27771
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27772
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27773
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-Inv-2023.27774
Searching KM on direction...
Found KM on Road 2059: ERS-122
CSG-In

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27841
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27842
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27843
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27844
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27845
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27846
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27847
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CSG-Inv-2023.27848
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2104: ERS-122
CS

Anexando as imagens

In [85]:
for reporting, images in tqdm(objects):
    for image in images:
        reporting_file = ReportingFile(
            created_by=user,
            reporting=reporting,
            description=image.name,
            km=reporting.km,
            point=reporting.point
        )
        reporting_file.save()
        image_io = BytesIO(image._data())
        im = Image.open(image_io)
        if im.mode in ("RGBA", "P"):
            im = im.convert("RGB")
        thumb_io = BytesIO()
        im.save(thumb_io, format='jpeg', quality=90)
        reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/190 [00:00<?, ?it/s]

In [23]:
reps = Reporting.objects.filter(company=company,occurrence_type=occurrence_type,created_at__gte='2023-5-10')
reps.count()

0

In [22]:
for a in reps:
    a.delete()